# Assignment 4: RNN

In [19]:
from src import RNN 
from src import preprocessing 
import importlib
importlib.reload(preprocessing)
importlib.reload(RNN)

<module 'src.RNN' from '/Users/annasanchezespunyes/Documents/GitHub/DD2424/lab4/src/RNN.py'>

In [22]:
read_data = load_data("/Users/annasanchezespunyes/Documents/GitHub/DD2424/lab4/data/goblet_book.txt")

In [23]:
# Sequence of input characters from the text
e = 0
sequence_length = 100

input_chars = read_data['data'][e:e+sequence_length]
target_chars = read_data['data'][e+1:e+sequence_length+1]

# Convert to one-hot encoding
#input_labels = self.encode(input_chars)
#target_labels = self.encode(target_chars)
